# Longitudinal GWAS Profiler

In [1]:
# start julia with `julia --track-allocation=user`

using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using ProfileCanvas

# simulate data
k = 0 # number of causal SNPs
qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 100, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2022,
    y_distribution = Bernoulli,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

# fit null
optm = QuasiCopula.fit!(qc_model,
    Ipopt.IpoptSolver(
        print_level = 5, 
        tol = 10^-3, 
        max_iter = 1000,
        accept_after_max_steps = 50,
        warm_start_init_point="yes", 
        limited_memory_max_history = 6, # default value
        hessian_approximation = "limited-memory",
        derivative_test="first-order"
    )
);

QuasiCopula.GWASCopulaVCModel(qc_model, G, check_grad=false)
Profile.clear_malloc_data() 
QuasiCopula.GWASCopulaVCModel(qc_model, G, check_grad=false)

[ Info: Precompiling QuasiCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
  ** incremental compilation may be fatally broken for this module **

[ Info: Precompiling ProfileCanvas [efd6af41-a80b-495e-886c-e51b0c7d77a3]


## Another profiler

In [5]:
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using Profile
using PProf

# simulate data
k = 0 # number of causal SNPs
qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 100, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2022,
    y_distribution = Bernoulli,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

# fit null
optm = QuasiCopula.fit!(qc_model,
    Ipopt.IpoptSolver(
        print_level = 5, 
        tol = 10^-3, 
        max_iter = 1000,
        accept_after_max_steps = 50,
        warm_start_init_point="yes", 
        limited_memory_max_history = 6, # default value
        hessian_approximation = "limited-memory",
        derivative_test="first-order"
    )
);

Profile.Allocs.@profile QuasiCopula.GWASCopulaVCModel(qc_model, G, check_grad=false)
Profile.Allocs.clear()
Profile.Allocs.@profile QuasiCopula.GWASCopulaVCModel(qc_model, G, check_grad=false)
PProf.Allocs.pprof()

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.


No errors detected by derivative checker.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       17
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:  

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Wtime = 1.3689272560056407
Qtime = 0.472063098999056
Rtime = 0.1671928159980418
grad_res_time = 0.2641863670001562
othertime = 0.282700454999634
scoretest_time = 0.005478075999999993


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


Wtime = 1.3791342600029874
Qtime = 0.47539721199927765
Rtime = 0.16744108699789634
grad_res_time = 0.2612310660002381
othertime = 0.2807036599995826
scoretest_time = 0.0005472530000000001


Analyzing 126 allocation samples... 100%|████████████████| Time: 0:00:00


"alloc-profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:62261
